In this notebook we will train LDA model on the full text of a job dataset ( cleaned jobs from Kaggle job recommendation challenge ) with 25 topics

# connect to drive

In [ ]:
#connect to drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# import

In [ ]:
!pip install pyLDAvis

In [ ]:
import pandas as pd
import ast
#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
from gensim.test.utils import datapath
from gensim import  models

#spacy
import spacy
#vis
import pyLDAvis
import pyLDAvis.gensim

# prepare data

In [ ]:
jobs=pd.read_csv("/content/drive/MyDrive/data/kaggle_recommend/all_jobs.csv")
jobs

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Unnamed: 0,JobID,City,State,Country,clean_title,clean_title_l,clean_full_text,text_non_stop,extracted_skills
0,0,457960,Raleigh,NC,US,data specialist,data specialist,data specialist functional area sales support ...,data specialist functional area sales support ...,"['data', 'sales', 'support', 'office', 'reloca..."
1,1,457962,Raleigh,NC,US,global infrastructure operations manager,global infrastructure operations manager,global infrastructure operations manager funct...,global infrastructure operations manager funct...,"['global', 'infrastructure', 'operations', 'se..."
2,2,457967,Raleigh,NC,US,investment analyst,investment analyst,investment analyst functional area accounting ...,investment analyst functional area accounting ...,"['investment', 'accounting', 'finance', 'offic..."
3,3,457998,Chandler,AZ,US,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr,back office medical assistant up to 12 hr b...,back office medical assistant 12 hr back offic...,"['office', 'office', 'clients', 'healthcare', ..."
4,4,457999,Buffalo,NY,US,medical biller,medical biller,medical biller company overview for over 15 ye...,medical biller company overview 15 years clien...,"['client', 'patient support', 'health safety',..."
...,...,...,...,...,...,...,...,...,...,...
309995,179995,948968,Baltimore,MD,US,magazines account executive,magazines account executive,magazines account executive account executive ...,magazines account executive account executive ...,"['magazines', 'magazines', 'product', 'adverti..."
309996,179996,949002,Spring Lake,MI,US,seasonal restaurant staff,seasonal restaurant staff,seasonal restaurant staff we are looking for e...,seasonal restaurant staff looking experienced ...,"['hospitality', 'servers', 'ess', 'servers', '..."
309997,179997,949034,Davenport,IA,US,freight loader,freight loader,freight loader freight handlers needed monday...,freight loader freight handlers needed monday ...,"['freight', 'freight', 'openings', 'freight', ..."
309998,179998,949834,Chicago,IL,US,senior manager field marketing,senior manager field marketing,senior manager field marketing organizationa...,senior manager field marketing organizational ...,"['field marketing', 'institutional', 'services..."


❗: there is **310001 rows**

# prepare to train model

In [ ]:
# first we need to split text to words (tokens)
jobs['tokens']=jobs['text_non_stop'].apply(lambda x: x.split())

In [ ]:
'''
Create a dictionary from the tokenized text
This is a mapping between the unique tokens encountered in your corpus and unique integer IDs
'''
dictionary = corpora.Dictionary(jobs['tokens'])

In [ ]:
'''
Corpus refers to the collection of documents to analyze using LDA.
Each document in the corpus is represented as a bag-of-words, which is a list containing the IDs (from the dictionary) of the words present in
that document, along with their frequencies (how many times each word appears)
'''
corpus = [dictionary.doc2bow(t) for t in jobs['tokens']]

# coherence & perplexity


In [ ]:
# Define a function to train LDA model, compute coherence score and perplexity
def train_lda_compute_metrics(num_topics, corpus, dictionary, texts):
    # Train LDA model
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=num_topics,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=10000,
                                           passes=20,
                                           alpha=0.01,
                                           eta=0.1)

    # Compute coherence score
    coherence_model = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
    coherence_score = coherence_model.get_coherence()

    # Compute perplexity
    perplexity = lda_model.log_perplexity(corpus)

    return lda_model, coherence_score, perplexity


In [ ]:
# Train LDA models with different numbers of topics and compute metrics
# for full text LDA we will only train on 25 to compare with skills-only model
num_topics_list = [25]
results = []
texts = jobs['tokens']
for num_topics in num_topics_list:
    lda_model, coherence_score, perplexity = train_lda_compute_metrics(num_topics, corpus, dictionary, texts)
    results.append({'num_topics': num_topics, 'coherence': coherence_score, 'perplexity': perplexity})
    temp_file = datapath(f"/content/drive/MyDrive/LDA_models/lda_model_{num_topics}_allText")
    lda_model.save(temp_file)

# Convert results to DataFrame for easier analysis
results_df = pd.DataFrame(results)

# Print the results
results_df

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,num_topics,coherence,perplexity
0,25,0.513138,-7.659146


# save model

In [ ]:
temp_file = datapath("/content/drive/MyDrive/LDA_models/lda_model_allText_25")
lda_model.save(temp_file)

# print top 10 words

In [ ]:
lda_model.print_topics(num_words=10)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(5,
  '0.012*"inventory" + 0.012*"warehouse" + 0.011*"driver" + 0.011*"work" + 0.011*"lift" + 0.010*"must" + 0.009*"truck" + 0.009*"ability" + 0.008*"able" + 0.008*"equipment"'),
 (16,
  '0.048*"sales" + 0.042*"marketing" + 0.031*"entry" + 0.030*"level" + 0.028*"management" + 0.019*"customer" + 0.018*"manager" + 0.016*"business" + 0.016*"service" + 0.011*"advertising"'),
 (2,
  '0.037*"food" + 0.026*"restaurant" + 0.025*"manager" + 0.018*"service" + 0.012*"customer" + 0.010*"quality" + 0.009*"managers" + 0.008*"team" + 0.008*"guest" + 0.008*"experience"'),
 (0,
  '0.036*"automotive" + 0.027*"service" + 0.019*"auto" + 0.014*"parts" + 0.014*"technician" + 0.014*"vehicle" + 0.013*"customer" + 0.012*"repair" + 0.010*"customers" + 0.009*"vehicles"'),
 (8,
  '0.015*"benefits" + 0.014*"company" + 0.014*"work" + 0.013*"insurance" + 0.010*"team" + 0.010*"opportunity" + 0.010*"career" + 0.009*"paid" + 0.009*"life" + 0.008*"dental"'),
 (22,
  '0.023*"business" + 0.014*"management" + 0.011*"exper

# Visualize

In [ ]:
import warnings
# Filter out warnings
warnings.filterwarnings("ignore")


pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, mds="mmds", R=30)
vis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
22    -0.031509  0.174541       1        1  10.080336
8     -0.154248 -0.080982       2        1   8.881021
7      0.080798  0.123054       3        1   7.683321
24     0.182099  0.199676       4        1   6.829596
17     0.063348  0.249638       5        1   5.966043
3     -0.149758 -0.176821       6        1   5.108136
13     0.335338  0.069347       7        1   5.027472
23    -0.307136  0.121080       8        1   4.866298
21     0.043111  0.035293       9        1   4.344275
14     0.274497  0.164568      10        1   4.334920
19     0.040152 -0.359697      11        1   4.155520
6      0.340581 -0.115171      12        1   4.013850
16    -0.267859  0.025033      13        1   3.540633
20     0.184742 -0.123494      14        1   3.298326
18     0.219816 -0.009985      15        1   3.228676
9     -0.139032  0.173223      16        1   3.169495
5      0.107992 -0.157814      17        1   3.073021
11     0.160343  0.341943      18        1   2.742780
4     -0.177161 -0.414293      19        1   1.651477
12    -0.036433  0.404815      20        1   1.606537
2     -0.266504  0.254378      21        1   1.583556
1     -0.396675 -0.067288      22        1   1.548657
15    -0.370754 -0.227203      23        1   1.380095
0     -0.020012 -0.265002      24        1   1.094541
10     0.284264 -0.338840      25        1   0.791422, topic_info=             Term           Freq          Total Category  logprob  loglift
119         sales  642400.000000  642400.000000  Default  30.0000  30.0000
892     marketing  199626.000000  199626.000000  Default  29.0000  29.0000
219       project  151010.000000  151010.000000  Default  28.0000  28.0000
36       customer  416094.000000  416094.000000  Default  27.0000  27.0000
455       service  382200.000000  382200.000000  Default  26.0000  26.0000
...           ...            ...            ...      ...      ...      ...
148          work    3296.282753  526529.016365  Topic25  -5.1315  -0.2344
415   opportunity    2784.406742  193390.545247  Topic25  -5.3003   0.5984
1333     programs    2390.120527   90991.139214  Topic25  -5.4530   1.1997
54     experience    2444.991295  734232.249136  Topic25  -5.4303  -0.8657
1195          may    2228.492564  118910.195421  Topic25  -5.5230   0.8621

[2079 rows x 6 columns], token_table=       Topic      Freq           Term
term                                 
66577      8  0.998640       "central
66578      8  0.998640            "cp
25471      8  0.998852         "every
34486      6  0.994970  "experienced"
19868      8  0.995752             "i
...      ...       ...            ...
1933      15  0.185210          youll
1933      24  0.015627          youll
9963      15  0.925981          youth
9963      23  0.073440          youth
86861     23  0.994895             yp

[9008 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[23, 9, 8, 25, 18, 4, 14, 24, 22, 15, 20, 7, 17, 21, 19, 10, 6, 12, 5, 13, 3, 2, 16, 1, 11])